# Introduction

As a project grows, you should <span style="color:lightgreen">***split code into multiple modules and then multiple files***</span>. <span style="color:lightgreen">***A package can contain multiple binary crates and optionally one library crate***</span>. As a package grows, you can extract parts into separate crates that become external dependencies